# Mathematical Optimization Lab 🛫

## Lecture 04: Gurobi exercises and an heuristic introduction 🐍

### By Samuele Lippolis
samuele.lippolis@studenti.units.it (write me here) \
samuelelippoli@gmail.com (write me here ONLY if I do not answer on the institutional mail)

### Remark
Throughout the notebook, there may be grammatical errors. I apologize for this. However, I preferred to create a long and comprehensive notebook with the risk of having grammatical errors rather than creating a concise notebook that might be ambiguous.

# Index
- Gurobi
  - Sudoku solution explanation
  - Facility location problem (FLP): exercise
  - Building artifical instances for FLP
  - Network Flow problem (NFP): exercise
  - Building artifical instances for NFP: exercise
- Heuristics introduction
  - Knapsack problem: density heuristic
  - Knapsack problem: random search


# Gurobi

# Index

  


### Sudoku solution explanation

Solve the following Sudoku and print the solution
$$
\begin{aligned}
&\begin{array}{|ccc|ccc|ccc|}
\hline 7 & . & 8 & 2 & 5 & . & . & . &  . \\
. & 4 &  . & 8 & . & . & 5 & . & .\\
5 & . & . & . & . & . & 6 & . & . \\
\hline
8 & . & . & 6 & . & . & . & 1 & 4 \\
. & 1 & 9 & . & . & . & 2 & 6 & . \\
2 & 6 & . & . & . & 1 & . & . & 5 \\
\hline
. & . & 1 & . & . & . & . & . & . \\
. & . & . & . & . & 8 & . & 5 & . \\
. & . & . & . & 3 & 4 & 7 & . & 9 \\
\hline
\end{array}
\end{aligned}
$$

In [2]:
# install xpress (the solver we will use)
!pip install gurobipy

# Lib
import numpy as np
import gurobipy as gb

# Sudoku
sudoku=[[ 7 ,"*", 8 , 2 , 5 ,"*","*","*","*"],
        ["*", 4 ,"*", 8 ,"*","*", 5 ,"*","*"],
        [ 5 ,"*","*","*","*","*", 6 ,"*","*"],
        [ 8 ,"*","*", 6 ,"*","*","*", 1 , 4 ],
        ["*", 1 , 9 ,"*","*","*", 2 , 6 ,"*"],
        [ 2 , 6 ,"*","*","*", 1 ,"*","*", 5 ],
        ["*","*", 1 ,"*","*","*","*","*","*"],
        ["*","*","*","*","*", 8 ,"*", 5 ,"*"],
        ["*","*","*","*", 3 , 4 , 7 ,"*", 9 ]]

# Model
model = gb.Model()
model.setParam('outputFlag', 0) # Avoid verbose output

rows = range(9)
columns = range(9)
numbers = range(9)

# Add var
# x_ijk = 1 if in position (i,j) there is the number k
X = model.addVars(
    [(i,j,k) for i in rows for j in columns for k in numbers], vtype = gb.GRB.BINARY
)

# Add constr
# Given a cell (i,j), you must have exactly one number
for i in rows:
    for j in columns:
        model.addConstr(
            gb.quicksum( X[i,j,k] for k in numbers) == 1
        )

# Given a row, you must have extacly 9 different numbers
# So, we ask two things: excactly 9 AND different

# 9 numbers in a row
for i in rows:
        model.addConstr(
            gb.quicksum( X[i,j,k] for k in numbers for j in columns) == 9
        )
        # 9 numbers are different = each number must appear exactly once in a row
        for k in numbers:
             model.addConstr(
                gb.quicksum( X[i,j,k] for j in columns) == 1
             )

# Symmetric concept on columns
for j in columns:
    model.addConstr(
         gb.quicksum( X[i,j,k] for k in numbers for i in rows) == 9
    )

    for k in numbers:
         model.addConstr(
              gb.quicksum( X[i,j,k] for i in rows) == 1
         )

# Moving along the matrix: first scan:
# 3x3 block top-left, second scan: 3x3 block at the top, one step right, and so on
# Moving trhough the blocks: first scanned element: top-left, second scan: top one step to the right, and son on
for ii in range(3):
  for jj in range(3):
    # Fix a block, I want that the sum of different numbers in all the block is exactly 9
    model.addConstr(
        gb.quicksum(X[ii*3 + i, jj*3 + j, k] for i in range(3) for j in range(3) for k in numbers) == 9
        )
    # Fix a block 3x3, fix a number, I want that number appears exacly once
    for k in numbers:
      model.addConstr(
          gb.quicksum(X[ii*3 + i, jj*3 + j, k] for i in range(3) for j in range(3)) == 1
          )

# Fix the number that already fill the sudoku
for i in rows:
    for j in columns:
        if sudoku[i][j] != "*":
            model.addConstr( X[i,j,sudoku[i][j] - 1] == 1)

# Solve
model.optimize()

# Print the solution
solution = np.zeros((9,9))
for i in rows:
    for j in columns:
        for k in numbers:
            if X[i,j,k].x == 1:
                solution[i,j] = k + 1

for row in sudoku:
    print(row)
print()
for row in solution:
    print(row)

[7, '*', 8, 2, 5, '*', '*', '*', '*']
['*', 4, '*', 8, '*', '*', 5, '*', '*']
[5, '*', '*', '*', '*', '*', 6, '*', '*']
[8, '*', '*', 6, '*', '*', '*', 1, 4]
['*', 1, 9, '*', '*', '*', 2, 6, '*']
[2, 6, '*', '*', '*', 1, '*', '*', 5]
['*', '*', 1, '*', '*', '*', '*', '*', '*']
['*', '*', '*', '*', '*', 8, '*', 5, '*']
['*', '*', '*', '*', 3, 4, 7, '*', 9]

[7. 3. 8. 2. 5. 6. 9. 4. 1.]
[1. 4. 6. 8. 7. 9. 5. 2. 3.]
[5. 9. 2. 4. 1. 3. 6. 7. 8.]
[8. 5. 7. 6. 9. 2. 3. 1. 4.]
[4. 1. 9. 3. 8. 5. 2. 6. 7.]
[2. 6. 3. 7. 4. 1. 8. 9. 5.]
[9. 8. 1. 5. 6. 7. 4. 3. 2.]
[3. 7. 4. 9. 2. 8. 1. 5. 6.]
[6. 2. 5. 1. 3. 4. 7. 8. 9.]


### Facility location problem (FLP): exercise

The Facility location problem: \
there are some clients and some facilities that can be used.\
Each possible facility has a different usage cost. \
Each facility has a annual good capacity. \
Each transport of good from a facility to a client has a different cost (for instance you can see it as the distance).
Each client has an annual demand.  

The optimisation problem is to choose a subset of the locations at
which to place facilities and then assign the clients to these
facilities as to minimise the total cost.

#### Compact formalization:

**Problem**:
* Facilities: $1,...,n (j)$
* Clients: $1,...,m (i)$
* $f_j$: cost of activating a facility in the position $j$
* $M_j$: annual capacity of the facility $j$
* $c_{ij}$: cost of serving the client $i$ from the facility $j$
* $d_i$: annual demand of the client $i$

**Decision variables:** \
There are two decision variables. \
Which are the choices?

**Objective and constraints:** \
Objective function:
* placement + transaport \
Constraint:
* satisfy the demand
* from a facility you can serve at most the capacity (but this is possible only if ... )

In [3]:
import gurobipy as gp

# FACILITY LOCATION PROBLEM:
def solve_facility_location(n, m, f, u, h, b):

  N = range(n) # n. of facility locations
  M = range(m) # n. of clients
  #-------------------------------------------------------------------------------

  # definition of the model
  model = gp.Model("capacitated_flp")
  model.setParam('outputFlag', 0) # Avoid verbose output

  # add variables to the model
  select = model.addVars(N, lb=0, ub=1, vtype=gp.GRB.BINARY, name="select_facilities")
  assign = model.addVars(N, M, lb=0.0, vtype=gp.GRB.CONTINUOUS, name='assign_good_to_each_location')

  # set obj function
  model.setObjective(gp.quicksum(f[j] * select[j] for j in N) +
                    gp.quicksum(h[i,j] * assign[i,j] for i in M for j in N), sense=gp.GRB.MINIMIZE)

  # add constraints:
  # 1) each client’s demand must be satisfied
  for i in M: # for a fixed col/client
      model.addConstr(gp.quicksum(assign[i,j] for j in N) == b[i], name=f"demand_{i}")

  # 2) annual capacity of each facility has to be respected
  for j in N: # for a fixed row/facility
      model.addConstr(gp.quicksum(assign[i,j] for i in M) - u[j] * select[j] <= 0, name=f"capacity_{j}")

  #-------------------------------------------------------------------------------

  # execute model
  model.optimize()

  # print solutions just in case status == gb.GRB.OPTIMAL
  if model.status == 2:
  # 1) which facilities have been activated:
    for facility in select.keys():
        if (abs(select[facility].x) > 1e-6):
            print(f"\n Activated location {facility}.")

    # 2) how much goods are supplied to each client from each facility (%):
    for customer, facility in assign.keys():
        if (abs(assign[customer, facility].x) > 1e-6):
            print(f"\n Client {customer} receives {round(assign[customer, facility].x, 2)} % of annual demand from facility {facility}.")

In [4]:
# SOLVE THE PROBLEM ON A SIMPLIFIED INSTANCE:

# input data
n = 3 # n. of facilities (rows)
m = 3 # n. of clients (cols)
u = {0: 100, 1: 150, 2: 125} # annual capacity of each facility (M)
b = {0: 80, 1: 120, 2: 100} # annual demand of each client (d)
f = {0: 10, 1: 15, 2: 20} # cost of activating each facility (f)
h = {(0,0): 5, (0,1): 8, (0,2): 3,
    (1,0): 7, (1,1): 6, (1,2): 4,
    (2,0): 5, (2,1): 6, (2,2): 7} # shipping costs of serving client i from location j (c)

# solve the constrained FLP
solve_facility_location(n=n, m=m, f=f, u=u, h=h, b=b)


 Activated location 0.

 Activated location 1.

 Activated location 2.

 Client 0 receives 80.0 % of annual demand from facility 2.

 Client 1 receives 75.0 % of annual demand from facility 1.

 Client 1 receives 45.0 % of annual demand from facility 2.

 Client 2 receives 100.0 % of annual demand from facility 0.


### Generate artificial instances

In [5]:
import random

def generate_instances(num_facilities,num_clients):
    f = {}
    u = {}
    h = {}
    b = {}
    for i in range(num_facilities):
        u[i] = random.randint(40, 80)
        f[i] = random.randint(u[i]-15, u[i]+15)

        for j in range(num_clients):
            h[(i,j)] = random.randint(1,10)

    for j in range(num_clients):
        b[j] = random.randint(20,40)

    return f,u,h,b

# Example
f,u,h,b = generate_instances(num_facilities=4, num_clients=7)
print("cost: ", f)
print("capacity: ", u)
print("arc cost: ", h)
print("demand: ",b)


cost:  {0: 86, 1: 44, 2: 49, 3: 67}
capacity:  {0: 75, 1: 45, 2: 59, 3: 65}
arc cost:  {(0, 0): 7, (0, 1): 8, (0, 2): 7, (0, 3): 2, (0, 4): 4, (0, 5): 2, (0, 6): 2, (1, 0): 3, (1, 1): 4, (1, 2): 9, (1, 3): 9, (1, 4): 6, (1, 5): 4, (1, 6): 6, (2, 0): 4, (2, 1): 2, (2, 2): 2, (2, 3): 10, (2, 4): 4, (2, 5): 9, (2, 6): 3, (3, 0): 8, (3, 1): 1, (3, 2): 9, (3, 3): 9, (3, 4): 7, (3, 5): 10, (3, 6): 7}
demand:  {0: 21, 1: 24, 2: 31, 3: 24, 4: 30, 5: 34, 6: 34}


In [6]:
# More specific
def generate_instances_2(
        num_facilities,num_clients, min_capacity, max_capacity, capacity_cost_deviation, min_edge_cost, max_edge_cost, min_demand, max_demand
        ):
    f = {}
    u = {}
    h = {}
    b = {}
    for i in range(num_facilities):
        u[i] = random.randint(min_capacity, max_capacity)
        f[i] = random.randint(u[i]-capacity_cost_deviation, u[i]+capacity_cost_deviation)

        for j in range(num_clients):
            h[(i,j)] = random.randint(min_edge_cost,max_edge_cost)

    for j in range(num_clients):
        b[j] = random.randint(min_demand,max_demand)

    return f,u,h,b

# Example
f,u,h,b = generate_instances_2(num_facilities = 10, num_clients = 30,
                               min_capacity = 40, max_capacity = 150,
                               capacity_cost_deviation = 20, min_edge_cost = 5,
                               max_edge_cost = 20, min_demand = 10, max_demand = 30)

print("cost: ", f)
print("capacity: ", u)
print("arc cost: ", h)
print("demand: ",b)

cost:  {0: 112, 1: 135, 2: 125, 3: 131, 4: 96, 5: 140, 6: 101, 7: 75, 8: 96, 9: 53}
capacity:  {0: 100, 1: 115, 2: 118, 3: 118, 4: 109, 5: 139, 6: 84, 7: 91, 8: 116, 9: 72}
arc cost:  {(0, 0): 13, (0, 1): 8, (0, 2): 18, (0, 3): 10, (0, 4): 5, (0, 5): 11, (0, 6): 12, (0, 7): 15, (0, 8): 19, (0, 9): 9, (0, 10): 10, (0, 11): 14, (0, 12): 19, (0, 13): 17, (0, 14): 18, (0, 15): 17, (0, 16): 20, (0, 17): 11, (0, 18): 9, (0, 19): 12, (0, 20): 19, (0, 21): 19, (0, 22): 18, (0, 23): 11, (0, 24): 19, (0, 25): 15, (0, 26): 13, (0, 27): 16, (0, 28): 13, (0, 29): 19, (1, 0): 11, (1, 1): 5, (1, 2): 5, (1, 3): 10, (1, 4): 12, (1, 5): 17, (1, 6): 18, (1, 7): 19, (1, 8): 9, (1, 9): 19, (1, 10): 16, (1, 11): 20, (1, 12): 7, (1, 13): 16, (1, 14): 20, (1, 15): 8, (1, 16): 10, (1, 17): 19, (1, 18): 8, (1, 19): 5, (1, 20): 15, (1, 21): 8, (1, 22): 5, (1, 23): 10, (1, 24): 6, (1, 25): 8, (1, 26): 7, (1, 27): 9, (1, 28): 7, (1, 29): 12, (2, 0): 7, (2, 1): 10, (2, 2): 8, (2, 3): 20, (2, 4): 12, (2, 5): 14, (2,

### Network Flow problem (NFP): exercise

A network is composed by a set of nodes (facilities) and arcs. \
Each node has a demand, the demand can be
* greater then zero (supplier node)
* equal to zero (transit node)
* less then zero (demand node)
Assumption: the network demand is zero (i.e., the sum of all the demands is equal to zero).

#### Formalization
* $V$: set of facilities
* $A$: set of arcs, where $e \in A \rightarrow e = (i,j)$ where $i$ and $j$ are two nodes.
* $b_i$: demand of the node i
* $h_{ij}$: arc flow capacity of arc (i,j)
* $c_{ij}$: cost per unit of arc (i,j)

#### Assumption
$$
\sum_{i \in V} b_i = 0
$$

#### Goal
* Each node demand must be satisfy
* You have to respect the arc capacity
* You have to minimize the cost of the transport
  
Try to find the decision variable, the constraints and the objective function without seeing the slides.



In [7]:
import gurobipy as gp

# NETWORK FLOW PROBLEM:
def solve_network_flow(V, A, b, h, c):

  # definition of the model
  model = gp.Model("nfp")
  model.setParam('outputFlag', 0) # Avoid verbose output

  # add variables to the model
  y = model.addVars(A, vtype=gp.GRB.CONTINUOUS, name='flow_per_each_arc')

  # set obj function
  model.setObjective(gp.quicksum(c[a] * y[a] for a in A), sense=gp.GRB.MINIMIZE)

  # add constraints:
  # 1) flow conservation constraint
  for i in V: # for each vertex/node
    entering_from_node = [arc[0] for arc in A if arc[1] == i] # entering
    exiting_from_node = [arc[1] for arc in A if arc[0] == i] # exiting
    model.addConstr(gp.quicksum(y[i,j] for j in exiting_from_node) -
                    gp.quicksum(y[j,i] for j in entering_from_node) == b[i],
                    name=f"flow_conservaion_{i}")

  # 2) capacity of each arc has to be respected
  for a in A: # for each vertex/node
      model.addConstr(y[a] <= h[a], name=f"capacity_{a}")

  #-------------------------------------------------------------------------------

  # execute model
  model.optimize()

  # print solutions just in case status == gb.GRB.OPTIMAL
  if model.status == 2:
    print(f"\n The optimal objective functions returns a value of {model.objVal}")
    # what is the flow assigned to each arc:
    for i, j in A:
      print(f"\n A flow of {round(y[i,j].x, 2)} is assigned to arc {i, j}.")
  else:
    print("no solution found")

In [8]:
# SOLVE THE PROBLEM ON A SIMPLIFIED INSTANCE:

# input data
V = [0, 1, 2, 3]  # Set of facilities
A = [(0,1), (0,3),
     (1,2),
     (2,3),
     (3,0), (3,2)]  # set of arcs
b = {0: 5, 1: -2, 2: -3, 3: 0}  # Node demands
h = {(0,1): 10, (0,3): 5,
     (1,2): 10,
     (2,3): 5,
     (3,0): 5, (3,2): 5} # arc flow capacity
c = {(0,1): 10, (0,3): 1,
     (1,2): 10,
     (2,3): 5,
     (3,0): 1, (3,2): 1} # arc flow unit cost

# solve the NFP
solve_network_flow(V=V, A=A, b=b, h=h, c=c)


 The optimal objective functions returns a value of 26.0

 A flow of 2.0 is assigned to arc (0, 1).

 A flow of 3.0 is assigned to arc (0, 3).

 A flow of 0.0 is assigned to arc (1, 2).

 A flow of 0.0 is assigned to arc (2, 3).

 A flow of 0.0 is assigned to arc (3, 0).

 A flow of 3.0 is assigned to arc (3, 2).


### Exercise: a function to build instances
Given the number of nodes, it must build
* an array for the nodes
* a list of tuples (i,j) where i and j are nodes indexes
* a node demand array, s.t. the sum of all the values is zero
* the arc flow capacity dictionary
* the edge flow capacity cost  
  

In [9]:
import random

def generate_instances_3(V, A, min_edge_cost, max_edge_cost, min_demand, max_demand):
    # Generate demand/supply at each node, enforcing sum=0 assumption
    b = {i: random.randint(min_demand, max_demand) for i in range(len(V)-1)}
    b[len(V)] = -sum(b.values())
    # Generate random arc/edge capacities
    h = {(i, j): random.randint(min_capacity, max_capacity) for (i, j) in A}
    # Generate random costs associated with each arc/edge
    c = {(i, j): random.randint(min_edge_cost, max_edge_cost) for (i, j) in A}
    return b, h, c

# A possible instance
V = [0, 1, 2, 3]  # Set of facilities
A = [(0,1), (1,2), (2,3), (3,0), (0,3), (3,2)]  # Set of arcs
min_demand = -10
max_demand = 10
min_capacity = 40
max_capacity = 150
min_edge_cost = 5
max_edge_cost = 20
# alternatively you can express one among cost or capacity in relative terms by means of
# a variable such as: capacity_cost_deviation = 20


# run a single instance randomly generated
b, h, c = generate_instances_3(V, A, min_edge_cost, max_edge_cost, min_demand, max_demand)

# print results
print("Node demands:", b)
print("Arc flow capacity:", h)
print("Edge flow cost:", c)


Node demands: {0: -3, 1: -4, 2: -2, 4: 9}
Arc flow capacity: {(0, 1): 91, (1, 2): 98, (2, 3): 74, (3, 0): 108, (0, 3): 94, (3, 2): 122}
Edge flow cost: {(0, 1): 17, (1, 2): 5, (2, 3): 17, (3, 0): 7, (0, 3): 5, (3, 2): 9}


# Heuristics introduction
A heuristic method is a procedure that aims to find solutions in a reasonable amount of time, often sacrificing optimality for efficiency. Unlike exact methods, heuristic methods provide a good solution quickly, but without the guarantee of optimality.

### Knapsack problem

#### Formalization
**Problem**
* $1,...,m$: projects
* $a_j$: cost of the project j
* $c_j$: value of the project j
* $b$: budget
* Goal: maximize the value, respecting the budget

**Decision variable:**
* $x_j$: $x_j = 1$ means that the do the j project

**Obj and constraints:**
$$
\sum_j c_j x_j
$$

$$
\sum_j a_j x_j \leq b
$$

$$
x_j \in \{0,1\}, \forall j \in \{1,...,m\}
$$

#### Exact method

In [10]:
# Exact method
def kanpsack_exact(a, c, b):
    if len(a) != len(c):
        return

    m = len(a)
    model = gb.Model("project_selection")
    model.setParam('outputFlag', 0) # Avoid verbose output

    # Decision variables
    x = model.addVars(m, vtype=gb.GRB.BINARY, name="x")

    # Objective function
    model.modelSense = gb.GRB.MAXIMIZE
    model.setObjective(
        gb.quicksum(c[j] * x[j] for j in range(m))
    )

    # Constraint: Budget constraint
    model.addConstr(
        gb.quicksum(a[j] * x[j] for j in range(m)) <= b,
        name="budget_constraint"
    )

    # Solve
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        print("Optimal solution found:")
        for j in range(m):
            if x[j].x > 0.5:  # Considered selected if x[j] >= 0.5
                print(f"Do project {(a[j],c[j])}.")
        print(f"Sum of values:{model.objVal}")
    else:
        print("No solution found.")


In [11]:
# A possible instance
costs = [10, 15, 20]  # Cost of each project
values = [5, 8, 10]  # Value of each project
budget = 25  # Budget

# Solve the problem
kanpsack_exact(a = costs, c = values, b = budget)

Optimal solution found:
Do project (10, 5).
Do project (15, 8).
Sum of values:13.0


#### Heuristic method

In [12]:
# Heuristic method

def knapsack_density(costs, values, budget):
    # Storing the original budget
    original_budget = budget

    # Build the density list
    value_density = []

    for i in range(len(costs)):
        value_density.append(
            (values[i]/costs[i],i)
            )

    value_density.sort(reverse=True)

    # Solve
    total_value = 0
    knapsack = []

    for item in value_density:
        if budget >= costs[item[1]]:
            knapsack.append((costs[item[1]],values[item[1]]))
            total_value += values[item[1]]
            budget -= costs[item[1]]
        elif budget <= 0:
            break

    return knapsack, total_value, (original_budget - budget)


In [13]:
# Example
selected_items, total_value, expenditure = knapsack_density(costs=costs, values=values, budget=budget)
print("Selected items:", selected_items)
print("Total value:", total_value)
print("Expenditure:", expenditure)

Selected items: [(15, 8), (10, 5)]
Total value: 13
Expenditure: 25


In [14]:
def knapsack_random(costs, values, budget, iterations=1000):
    # Storing the original budget
    original_budget = budget

    # Initialize best solution
    best_solution = []
    best_value = 0
    best_budget = 0

    for _ in range(iterations):
        current_solution = []
        current_value = 0
        current_budget = budget
        indexes = [i for i in range(len(costs))]

        # Randomly select items until budget is exhausted
        for _ in range(len(costs)):
            i = random.choice(indexes)
            if current_budget >= costs[i]:
                current_solution.append((costs[i], values[i]))
                current_value += values[i]
                current_budget -= costs[i]
                indexes.remove(i)


        # Update best solution if current solution is better
        if current_value > best_value:
            best_solution = current_solution
            best_value = current_value
            best_budget = current_budget

    return best_solution, best_value, (original_budget - best_budget)


In [15]:
# Example of the inner block
indexes = [i for i in range(10)]
for _ in range(4):
    i = random.choice(indexes)
    print(i)
    indexes.remove(i)
print(indexes)

3
5
4
6
[0, 1, 2, 7, 8, 9]


In [16]:
# Example
selected_items, total_value, expenditure = knapsack_random(costs=costs, values=values, budget=budget)
print("Selected items:", selected_items)
print("Total value:", total_value)
print("Expenditure:", expenditure)

Selected items: [(10, 5), (15, 8)]
Total value: 13
Expenditure: 25


#### Comparison on big instances

In [17]:
import numpy as np

# parameter -> budget 30, size = 10 AND budget 3000, size = 1000 AND budget 300000, size = 100000
budget = 30
size = 10


# Generate random numbers from a Gaussian distribution with mean equal to random_numbers[i]
np.random.seed(0)
costs = np.random.normal(loc=10, scale=3, size=size)
costs = np.round(costs) # to make them integers
costs = np.clip(costs, a_min=1, a_max=None) # to make the minimum = 1

# Generate an array of 100 items with Gaussian distribution around random_numbers[i]
values = []
for i in range(size):
    value = np.random.normal(loc=costs[i], scale=2)
    value = int(max(round(value), 1))  # Ensure item_value is an integer and >= 1
    values.append(value)

# Print the generated items
print(budget)
print(costs)
print(values)


30
[15. 11. 13. 17. 16.  7. 13. 10. 10. 11.]
[15, 14, 15, 17, 17, 8, 16, 10, 11, 9]


In [18]:
selected_items, total_value, expenditure = knapsack_density(costs=costs, values=values, budget=budget)
print("Selected items:", selected_items)
print("Total value:", total_value)
print("Expenditure:", expenditure)

Selected items: [(np.float64(11.0), 14), (np.float64(13.0), 16)]
Total value: 30
Expenditure: 24.0


In [19]:
selected_items, total_value, expenditure = knapsack_random(costs=costs, values=values, budget=budget, iterations=100)
print("Selected items:", selected_items)
print("Total value:", total_value)
print("Expenditure:", expenditure)

Selected items: [(np.float64(13.0), 16), (np.float64(10.0), 11), (np.float64(7.0), 8)]
Total value: 35
Expenditure: 30.0


In [20]:
kanpsack_exact(a = costs, c = values, b = budget)

Optimal solution found:
Do project (np.float64(7.0), 8).
Do project (np.float64(13.0), 16).
Do project (np.float64(10.0), 11).
Sum of values:35.0


## Conclusion

On small instances Gurobi is able to find the optimum, so it will be likely better than the heuristic (in terms of effectivness).
On the other hand, with big instances Gurobi is not able to use an excact approach sometimes, so, it will also use an herustic method.  